In [23]:
# importing mdules that are used in the code
import re
import string
from collections import Counter 
import numpy as np 


In [42]:
# opening file and read and w+ is to split the sentences in word by word
def myfile(file_path):
   with open(file_path, "r",encoding="utf-8") as file: # the file that is used here is utf-8 type , you can use your own
    lines = file.readlines()
    words = []
    for line in lines:
        words += re.findall(r'\w+', line.lower())

    return words

In [25]:
words = myfile("./book.txt") # the file name "book.txt" used here can be changed
word_count = len(words)
message = "The corpus contains " + str(word_count) + " total words."
print(message)

The corpus contains 8338 total words.


In [26]:
my_words = set(words)
print(len(my_words))

2083


In [27]:
word_counts = Counter(words)
print(word_counts["the"])

483


In [28]:
total_word= float(sum(word_counts.values()))
probability = {word:word_counts[word]/total_word for word in word_counts.keys() }
print(probability["he"],probability["the"])

0.010554089709762533 0.057927560566082996


In [29]:
def split(word):
    return [(word[:i], word[i:]) for i in range(len(word) + 1)]

def delete(word):
    return [l + r[1:] for l, r in split(word) if r]

def swap(word):
    return [l + r[1] + r[0] + r[2:] for l, r in split(word) if len(r) > 1]

def insert(word):
    letters = string.ascii_lowercase
    return [l + c + r for l, r in split(word) for c in letters]
def replace(word):
    letters = string.ascii_lowercase
    return [l + c + r[1:] for l, r in split(word) for c in letters if r]

In [30]:
 def editing(word):
    return set(delete(word) + swap(word) + replace(word) + insert(word))

#print(editing("have"))

In [31]:
def editing_2(word):
    return set(e2 for e1 in edit1(word) for e2 in edit1(e1))

In [32]:
def correct_spelling(word, vocabulary, word_probabilities):
    if word in vocabulary:
        print(f"{word} is already correctly spelled")
        return

    suggestions = editing(word) or editing_2(word) or [word]
    best_guesses = [w for w in suggestions if w in vocabulary]
    return [(w, word_probabilities[w]) for w in best_guesses]

In [33]:
word = "ihe"
corrections = correct_spelling(word, my_words, probability)

if corrections:
  print(corrections)
  probs = np.array([c[1] for c in corrections])
  best_ix = np.argmax(probs)
  #correct = corrections[best_ix][0]
  #print(f"{correct} is suggested for {word}")
  #print(correct)

[('the', 0.057927560566082996), ('he', 0.010554089709762533)]


In [45]:
class auto_correct(object):

  def __init__(self,file_path):
    with open(file_path, "r",encoding="utf-8") as file:
      lines = file.readlines()
      words = []
      for line in lines:
        words += re.findall(r'\w+', line.lower())

    self.vocabulary = set(words)
    self.word_counts = Counter(words)
    total_words = float(sum(self.word_counts.values()))
    self.word_probas = {word: self.word_counts[word] / total_words for word in self.vocabs}

  def editing(self, word):
    letters = string.ascii_lowercase
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [l + r[1:] for l,r in splits if r]
    swaps = [l + r[1] + r[0] + r[2:] for l, r in splits if len(r)>1]
    replaces = [l + c + r[1:] for l, r in splits if r for c in letters]
    inserts = [l + c + r for l, r in splits for c in letters] 

    return set(deletes + swaps + replaces + inserts)

  def editing(self, word):
    return set(e2 for e1 in self.editing(word) for e2 in self.editing(e1))

  def check(self, word):
    my_words = self.editing(word) or self.editing_2(word) or [word]
    valid_words = [w for w in my_words if w in self.vocabulary]
    return sorted([(c, self.word_probas[c]) for c in valid_words], key=lambda tup: tup[1], reverse=True)


In [46]:
checker = SpellChecker("./book.txt")# a text file saved 
checker.check("sentense")#one word from the textfile called book.text

[('sentence', 0.00023986567522187575)]